In [30]:
import lmfit
from lmfit import minimize, Parameters, fit_report
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import Analysis
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
def sigmoid(x,v0):
    return 1.0 / (1 + np.exp(np.array((-x+v0)/0.1,dtype='float')))


def residual(params, x, y, eps_data):
    amp = params['amp']
    phase = params['phase']
    delta_phase = params['delta_phase']
    freq = params['frequency']
    slope = params['slope']
    height = params['height']
    v0 = params['v0']
    model_r = height + amp * np.sin(np.array(x*freq+ phase,dtype='float')) + slope * (x - v0)
    model_l = height + amp * np.sin(np.array(x*freq + (phase+delta_phase*np.pi),dtype='float')) + slope * (x - v0)
    
    out_r = (y-model_r)*sigmoid(x,v0)
    out_l = (y-model_l)*(1.0-sigmoid(x,v0))
    
    return np.array((out_r+out_l)/eps_data,dtype='float')




In [68]:
def search(seq, t):  
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max <= min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [4]:
def plot_result(min_result):
    params = min_result.params
    x = np.arange(0,30,30.0/1000)[::-1]*-1.0
    amp = params['amp']
    phase = params['phase']
    delta_phase = params['delta_phase']
    freq = params['frequency']
    slope = params['slope']
    height = params['height']
    v0 = params['v0']
    model_r = height + amp * np.sin(np.array(x*freq+ phase,dtype='float')) + slope * (x - v0)
    model_l = height + amp * np.sin(np.array(x*freq + (phase+delta_phase*np.pi),dtype='float')) + slope * (x - v0)
    plt.plot(x,model_r,c='r')
    plt.plot(x,model_l,c='k')

In [80]:
def regression(x_min,x_max,data,plot=False):
    
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1
    
    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]
    
    a = (np.percentile(y,99)-np.percentile(y,1))/2.0
    
    params = Parameters()
    params.add('amp', value = a, min = a*0.8, max = a*1.2)
    h = np.mean(y)
    params.add('height', value = h, min = h*0.99, max = h*1.01)
    params.add('phase', value = 0.0, min = 0.0, max = 2*np.pi)
    params.add('delta_phase', value = np.random.uniform(0.1,1.9), min = 0.0, max = 2.0)
    params.add('frequency', value = 0.63, min = 0.6, max = 0.76)
    params.add('slope',value = 0.0, min = -0.01, max = 0.01)
    params.add('v0', value = (x_min+x_max)/2.0, vary = False)
    
    
    
    eps_data = 0.1
    
    out = minimize(residual, params, args=(x, y, eps_data))
    
    if plot:
        params = out.params
        amp = params['amp']
        phase = params['phase']
        delta_phase = params['delta_phase']
        freq = params['frequency']
        slope = params['slope']
        height = params['height']
        v0 = params['v0']
        model_r = height + amp * np.sin(np.array(x*freq+ phase,dtype='float')) + slope * (x - v0)
        model_l = height + amp * np.sin(np.array(x*freq + (phase+delta_phase*np.pi),dtype='float')) + slope * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    
    return out    


In [35]:
def multi_regression(x_min,x_max,data,n=20):
    red_chi_sq = np.inf
    result_hold = lmfit.minimizer.MinimizerResult()
    for _ in range(n):
        result = regression(x_min,x_max,data)
        if result.success:
            if result.redchi < red_chi_sq:
                red_chi_sq = result.redchi
                result_hold = result
    return result_hold

In [6]:
run = Analysis.generate_2014()

In [81]:
test_reg = regression(-31.020389999999999, -25.020389999999999,run.prune_data[0],True)

In [77]:
test_multi_reg = multi_regression(-31.020389999999999, -25.020389999999999, run.prune_data[0])

In [79]:
run.prune_data[0].plot()
plot_result(test_multi_reg)

In [100]:
def sweep_regression(data,d=6.0,c=20,n=20):
    out = list()
    x_max = max(data.x)
    x_min = min(data.x)
    step = (x_max-x_min-2*d)/c
    for i in range(c):
        center = x_min + d +  i*step
        low = center - d
        high = center + d
        print(low,high)
        result = multi_regression(low, high, data,n)
        out.append({'result':result, 'x_min':low, 'x_max':high})
    return out
    
    

In [101]:
test_sweep_reg = sweep_regression(run.prune_data[0])

(-31.020389999999999, -19.020389999999999)
(-30.266910599999999, -18.266910599999999)
(-29.513431199999999, -17.513431199999999)
(-28.7599518, -16.7599518)
(-28.0064724, -16.0064724)
(-27.252993, -15.252993)
(-26.4995136, -14.4995136)
(-25.746034199999997, -13.746034199999997)
(-24.992554800000001, -12.992554800000001)
(-24.239075399999997, -12.239075399999997)
(-23.485596000000001, -11.485596000000001)
(-22.732116599999998, -10.732116599999998)
(-21.978637199999998, -9.9786371999999979)
(-21.225157799999998, -9.2251577999999981)
(-20.471678399999998, -8.4716783999999983)
(-19.718198999999998, -7.7181989999999985)
(-18.964719599999999, -6.9647195999999987)
(-18.211240199999999, -6.2112401999999989)
(-17.457760799999999, -5.4577607999999991)
(-16.704281399999999, -4.7042813999999993)


In [102]:
for i in test_sweep_reg:
    plt.figure()
    run.prune_data[0].plot()
    
    plot_result(i['result'])
    plt.xlim(i['x_min'],i['x_max'])

In [103]:
c = 1
for i in test_sweep_reg:
    print(c)
    c+=1
    print(i['result'].params['delta_phase'],i['result'].redchi)

1
(<Parameter 'delta_phase', value=1.8443266044570585 +/- 0.0157, bounds=[0.0:2.0]>, 0.3014174544476404)
2
(<Parameter 'delta_phase', value=1.8269587644432685 +/- 0, bounds=[0.0:2.0]>, 0.33533717465095381)
3
(<Parameter 'delta_phase', value=1.8583547291349172 +/- 0.00963, bounds=[0.0:2.0]>, 0.43747412840359495)
4
(<Parameter 'delta_phase', value=1.6302391491020967 +/- 0.0127, bounds=[0.0:2.0]>, 0.42730073651054562)
5
(<Parameter 'delta_phase', value=1.5890204594715573 +/- 0, bounds=[0.0:2.0]>, 0.23805153250134703)
6
(<Parameter 'delta_phase', value=1.5977134429594164 +/- 0.0184, bounds=[0.0:2.0]>, 0.24234482748291467)
7
(<Parameter 'delta_phase', value=1.612591831044015 +/- 0.0154, bounds=[0.0:2.0]>, 0.38663554257128946)
8
(<Parameter 'delta_phase', value=1.5009675370344013 +/- 0.0155, bounds=[0.0:2.0]>, 0.46133179795709051)
9
(<Parameter 'delta_phase', value=1.3343169369983947 +/- 0.016, bounds=[0.0:2.0]>, 0.39700525154157668)
10
(<Parameter 'delta_phase', value=1.4919903060082436 +/-